In [1]:
import socket
import sys
import requests 
import requests_oauthlib
import json
import time

# CONSUMER_KEY = "L0wvzJIRsC4AgOCB9Mng4z5OP"
# CONSUMER_SECRET = "MaSEsGLADbvQbPEUIINxF0IMx99GBDay6KV3xc5tqzvZ6G7U1X " 
# ACCESS_TOKEN = "1052075882620301312-wvZyyn4PDLaPjzgwdvX8LQD7Nu6X1R" 
# ACCESS_SECRET = "Reg1bUQJ7fsk2V3pnJ1qodqn1m17mDQYfSGYQ0GLyDX6I"

ACCESS_TOKEN = '1048005627816169473-5tJdLYafXOfCUaHWDFtcqjCvBRdU90'
ACCESS_SECRET = 'iIQRVbZTN625Zn1xRPwDKkOH8Mk4xqrAEn8ICb39CTIeY'
CONSUMER_KEY = 'iTrUDoKffZNLfSYvc9ctDXGJZ'
CONSUMER_SECRET = 'XEuXkw8AesuyxYaiPG7CSI1r0IhuXhcN8vvTYyimrCoAvMShU4'

my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In [2]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data]) 
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

def send_tweets_to_spark(http_resp, tcp_connection): 
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line) 
            tweet_text = full_tweet['text']
            print(tweet_text) 
            tcp_connection.send((tweet_text + '\n').encode())
        except:
            e = sys.exc_info()[0]
            print(e)

In [3]:
TCP_IP = "localhost"
TCP_PORT = 10005
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
# try bind the port
for i in range(30):
    try:
        s.bind((TCP_IP, TCP_PORT))
        break
    except:
        print("connecting...")
        time.sleep(3)
if(i == 29):
    raise Exception("Can not bind the port, port in use")
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("conn = "+ str(conn))
print("addr = " + str(addr))
print("Connected... Starting getting tweets.") 
resp = get_tweets()
# print("get_tweets finished")
# send_tweets_to_spark(resp, conn)
# print("send_tweets_to_spark finished")

Waiting for TCP connection...
conn = <socket.socket fd=46, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 10005), raddr=('127.0.0.1', 33270)>
addr = ('127.0.0.1', 33270)
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>


In [4]:
s.close()